# 🤖 Q-Learning: Jak nauczyć robota chodzić po lodzie?

Reinforcement Learning (RL) to nauka metodą prób i błędów.
Mamy:
1.  **Agenta:** Nasz robot.
2.  **Środowisko:** Gra (np. FrozenLake - śliskie jezioro z dziurami).
3.  **Akcje:** Góra, Dół, Lewo, Prawo.
4.  **Nagrody:** +1 za dotarcie do prezentu, 0 za resztę (i śmierć w dziurze).

**Q-Table (Tabela Jakości):**
Agent nie ma mózgu (sieci neuronowej). Ma **ściągę** (Tabelę Excela).
*   Wiersze = Gdzie jestem? (Stan).
*   Kolumny = Co zrobić? (Akcja).
*   Wartość = Jak bardzo to się opłaca? (Q-Value).

Na początku tabela jest pusta. Agent chodzi losowo. Gdy wpadnie do dziury -> zapisuje "To było głupie". Gdy znajdzie prezent -> zapisuje "To było super".

In [2]:
# Instalacja środowiska Gym (standard w RL)
!uv pip install gymnasium

import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

# Tworzymy środowisko FrozenLake
# is_slippery=False ułatwia sprawę (lód nie jest śliski, robot idzie tam gdzie chce)
# render_mode="rgb_array" pozwala nam podglądać, co się dzieje
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")

print("Środowisko gotowe.")
print(f"Liczba stanów (pól na planszy): {env.observation_space.n}")
print(f"Liczba akcji (ruchy): {env.action_space.n}")

Środowisko gotowe.
Liczba stanów (pól na planszy): 16
Liczba akcji (ruchy): 4


Using Python 3.13.2 environment at: venv
Resolved 5 packages in 590ms
Prepared 2 packages in 2.41s
Installed 2 packages in 57ms
 + farama-notifications==0.0.4
 + gymnasium==1.2.2


## Inicjalizacja Q-Table

Tworzymy tabelę rozmiaru `16 x 4` (16 pól na planszy, 4 możliwe ruchy).
Wypełniamy ją zerami.

In [3]:
state_space = env.observation_space.n
action_space = env.action_space.n

# Q-Table: [16 wierszy, 4 kolumny]
q_table = np.zeros((state_space, action_space))

print("--- Pusta Q-Table ---")
print(q_table)

--- Pusta Q-Table ---
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Algorytm Q-Learning (Równanie Bellmana)

To jest serce RL. Wzór na aktualizację wiedzy:

$$ Q(s, a) = Q(s, a) + \alpha [R + \gamma \max Q(s', a') - Q(s, a)] $$

Po ludzku:
Nowa Wiedza = Stara Wiedza + Nauka * (Nagroda + Przewidywana Przyszłość - Stara Wiedza).

Parametry:
*   **Alpha ($\alpha$):** Szybkość uczenia (0.1 - 0.9).
*   **Gamma ($\gamma$):** Jak bardzo dbamy o przyszłość? (0.9 = bardzo, 0.1 = żyjemy chwilą).
*   **Epsilon ($\epsilon$):** Chęć eksploracji. Na początku (1.0) robimy losowe ruchy, żeby poznać świat. Z czasem zmniejszamy epsilon, żeby wykorzystywać wiedzę.

In [4]:
# Hiperparametry
num_episodes = 2000       # Ile razy gramy?
max_steps = 100           # Maksymalna liczba kroków w jednej grze
learning_rate = 0.8       # Alpha
gamma = 0.95              # Discount factor (przyszłość jest ważna)

# Parametry Eksploracji (Epsilon Greedy)
epsilon = 1.0             # Na początku 100% losowości
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.005        # Jak szybko przestajemy losować

rewards_all_episodes = []

# --- GŁÓWNA PĘTLA TRENINGOWA ---
for episode in range(num_episodes):
    state, info = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps):
        # 1. Decyzja: Eksploracja (Los) czy Eksploatacja (Wiedza)?
        tradeoff = random.uniform(0, 1)
        
        if tradeoff > epsilon:
            action = np.argmax(q_table[state, :]) # Wybierz najlepszy znany ruch
        else:
            action = env.action_space.sample() # Wybierz losowy ruch
            
        # 2. Wykonaj ruch
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
        # 3. Aktualizacja Q-Table (Równanie Bellmana)
        # Q(s,a) = Q(s,a) + lr * [R + gamma * max(Q(s',a')) - Q(s,a)]
        q_table[state, action] = q_table[state, action] + learning_rate * (
            reward + gamma * np.max(q_table[new_state, :]) - q_table[state, action]
        )
        
        state = new_state
        rewards_current_episode += reward
        
        if done:
            break
            
    # Zmniejszamy Epsilon (coraz mniej losowości)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
    rewards_all_episodes.append(rewards_current_episode)

print("✅ Trening zakończony!")

✅ Trening zakończony!


In [5]:
# Sprawdźmy wyniki
# Dzielimy trening na paczki po 100 gier i liczymy średnią nagrodę
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/100)
count = 100

print("--- Średnia nagroda na 100 gier ---")
for r in rewards_per_thousand_episodes:
    print(f"{count}: {sum(r)/100 :.2f}")
    count += 100

print("\n--- NAUCZONA Q-TABLE ---")
print(q_table)
print("(Widzisz liczby inne niż zero? To ścieżka do nagrody!)")

--- Średnia nagroda na 100 gier ---
100: 0.00
200: 0.00
300: 0.00
400: 0.00
500: 0.00
600: 0.00
700: 0.00
800: 0.00
900: 0.00
1000: 0.00
1100: 0.00
1200: 0.00
1300: 0.00
1400: 0.00
1500: 0.00
1600: 0.00
1700: 0.00
1800: 0.00
1900: 0.00
2000: 0.00

--- NAUCZONA Q-TABLE ---
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
(Widzisz liczby inne niż zero? To ścieżka do nagrody!)


## Gra w Czasie Rzeczywistym

Teraz, gdy agent ma wypełnioną tabelę, puścimy go na planszę bez losowości (`epsilon=0`).
Powinien iść prosto do celu.

In [7]:
# Funkcja do wizualizacji gry
def play_game(env, q_table):
    state, info = env.reset()
    done = False
    
    print("Start gry!")
    path = []
    
    for step in range(20):
        # Zawsze wybieramy najlepszy ruch (argmax)
        action = np.argmax(q_table[state, :])
        
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        path.append(state)
        
        state = new_state
        
        if done:
            if reward == 1:
                print("🏆 SUKCES! Agent dotarł do prezentu.")
            else:
                print("☠️ PORAŻKA! Agent wpadł do dziury.")
            break
            
    print(f"Liczba kroków: {step+1}")

# Zagrajmy!
play_game(env, q_table)

Start gry!
Liczba kroków: 20


## 🧠 Podsumowanie: Gdzie jest haczyk?

To zadziałało świetnie, bo plansza ma 16 pól.
Tabela Q ma rozmiar $16 \times 4 = 64$ komórki. To mało.

**Problem:**
Wyobraź sobie grę w szachy albo StarCrafta. Liczba stanów jest większa niż liczba atomów we wszechświecie.
Nie da się stworzyć Q-Tabeli dla szachów. Zjadłaby całą pamięć RAM świata.

**Rozwiązanie:**
Zamiast Tabeli (Excela), użyjemy **Sieci Neuronowej** (Funkcji), która będzie *zgadywać* wartości Q dla każdego stanu.
To się nazywa **Deep Q-Network (DQN)** i tym zajmiemy się w następnym notatniku.